In [ ]:
import pandas as pd

def csv_to_jsonl(csv_path, jsonl_path):
    df = pd.read_csv(csv_path)
    qa_list = df.to_dict(orient='records')
    save_multiple_qa_to_jsonl(qa_list, jsonl_path)

csv_to_jsonl("data.csv", "training_dataset.jsonl")


In [9]:
import json

def save_qa_to_jsonl(question, answer, filepath):
    """
    Save a question-answer pair into a JSONL file, preserving formatting.
    
    Args:
        question (str): The natural language question or prompt.
        answer (str): The code snippet (answer).
        filepath (str): Path to the .jsonl file.
    """
    data = {
        "question": question,
        "answer": answer
    }
    
    with open(filepath, 'a', encoding='utf-8') as f:
        json_line = json.dumps(data, ensure_ascii=False)
        f.write(json_line + '\n')

if __name__ == "__main__":
    q = """Allow ARP packets to flood across all ports to enable address resolution"""
    a = """package network.flow

default decision := "drop"

decision := "flood" if {
    input.protocol == "arp"
    input.action == "flood"
}
"""
    
    save_qa_to_jsonl(q, a, "training_dataset.jsonl")


In [ ]:
import requests
import json

ITM_url = "https://7708-35-184-143-188.ngrok-free.app/ask"
payload = {"question": "Servers are not allowed to expose the 'telnet' protocol"}
policyresponse = requests.post(ITM_url, json=payload)

opa_check_url = "http://fyp-opa-1:8181/v1/polices/" + str(random.randint(1000, 9999))
response = requests.put(url=opa_check_url, data=policyresponse, timeout=5)



{"question": "Servers are not allowed to expose the 'telnet' protocol"}


------
## Check translation acc + time translation

In [11]:
import requests
import json
import random
import time

ITM_url = "https://e259-34-57-224-94.ngrok-free.app/ask"
opa_url_base = "http://localhost:8181/v1/polices/"
success_count = 0
total_time = 0
timed_requests = 0

with open("testing_data.txt", "r") as f:
    questions = [line.strip() for line in f if line.strip()]

policy_id = 1000

for question in questions:
    payload = {"question": question}

    try:
        start_time = time.time()  # Start timing

        policy_response = requests.post(ITM_url, json=payload)
        policy_response.raise_for_status()

        end_time = time.time()  # End timing
        elapsed_time = end_time - start_time
        total_time += elapsed_time
        timed_requests += 1

        # Create a unique OPA policy name
        opa_check_url = opa_url_base + str(policy_id)

        response = requests.put(url=opa_check_url, data=policy_response, timeout=5)

        if response.status_code == 200:
            success_count += 1
        else:
            print(f"OPA error {response.status_code}: {response.text}")

    except Exception as e:
        print(f"Error with question '{question}': {e}")

    policy_id += 1

average_time = total_time / timed_requests if timed_requests > 0 else 0

print(f"Acc of translation: {success_count/timed_requests *100}\n")
print(f"Average response time for translation requests: {average_time:.5f} seconds")


OPA error 404: 404 page not found

OPA error 404: 404 page not found

OPA error 404: 404 page not found

OPA error 404: 404 page not found

OPA error 404: 404 page not found

OPA error 404: 404 page not found

OPA error 404: 404 page not found

OPA error 404: 404 page not found

OPA error 404: 404 page not found

OPA error 404: 404 page not found

OPA error 404: 404 page not found

OPA error 404: 404 page not found

OPA error 404: 404 page not found

OPA error 404: 404 page not found

OPA error 404: 404 page not found

OPA error 404: 404 page not found

OPA error 404: 404 page not found

OPA error 404: 404 page not found



KeyboardInterrupt: 

-------
## Check intent detection acc

In [48]:
import requests
import json

rasa_parse_url = "http://localhost:5005/model/parse"
success_count = 0
count = 0
with open("testing_data.txt", "r") as f:
    questions = [line.strip() for line in f if line.strip()]


for question in questions:
    payload = {
        "text": question,
        "message_id": "b2831e73-1407-4ba0-a861-0f30a42a2a5a"
    }

    try:
        response = requests.post(url=rasa_parse_url, json=payload, timeout=5)
        if response.status_code == 200 and response.json()["intent"]["name"] == "network_action":
            success_count += 1
            count += 1
        else:
            print(f"Non-200 response for '{question}': {response.status_code} - {response.text}")
    except Exception as e:
        print(f"Error with request '{question}': {e}")


print(f"Acc of intent detection: {success_count/count * 100}%")


Acc of intent detection: 100.0%
